In [49]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time

# class Crawling:
#     def get_year_data(self)

In [70]:
# get_recall_manufacturer_data.py
class Recall_Manufacturer_Data:
    def get_manufacturer_data(self):
        url = 'https://www.car.go.kr/rs/stats/rcList.do'
        driver = webdriver.Chrome()
        driver.get(url)
        search_year = ['2022','2023','2024']
        manufacturer_data = []
        for i in search_year:
            fromyear = Select(driver.find_element(By.XPATH, '//*[@id="recallYear"]'))
            fromyear.select_by_value(f'{i}')
            time.sleep(1)
            
            button = driver.find_element(By.XPATH, '//*[@id="srchBtn"]')
            button.click()
            time.sleep(1)
            
            button = driver.find_element(By.XPATH, '//*[@id="openTwoBtn"]')
            button.click()
            time.sleep(1)
            
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            datas = soup.select('#openTrTwo > tr')
            
            for tr in datas:
                values = [i+'년'] + [td.get_text(strip=True) for td in tr.find_all(["th", "td"])]
                manufacturer_data.append(values)
        
        driver.quit()
        return manufacturer_data
    
    def manufacturer_data_preprocessing(self, manufacturer_data):
        manufacturer_df = pd.DataFrame(data=manufacturer_data, columns=['recall_year','recall_month','car_model','car_num'])
        manufacturer_df = manufacturer_df.dropna()
        manufacturer_df['recall_month'] = manufacturer_df['recall_month'].str.replace('소계 +소계 -','총합')
        manufacturer_df['car_num'] = manufacturer_df['car_num'].str.replace(',','').astype(int)
        return manufacturer_df

    def get_manufacturer_df(self):
        return self.manufacturer_df

    def __init__(self):
        self.manufacturer_df = self.manufacturer_data_preprocessing(self.get_manufacturer_data())

In [65]:
# get_recall_year_data.py
class Recall_Year_Data:
    def get_year_data(self):
        url = 'https://www.car.go.kr/rs/stats/rcList.do'
        driver = webdriver.Chrome()
        driver.get(url)
        search_year = ['2022','2023','2024']
        year_data = []
        for i in search_year:
            fromyear = Select(driver.find_element(By.XPATH, '//*[@id="recallYear"]'))
            fromyear.select_by_value(f'{i}')
            time.sleep(1)
            
            button = driver.find_element(By.XPATH, '//*[@id="srchBtn"]')
            button.click()
            time.sleep(1)
            
            button = driver.find_element(By.XPATH, '//*[@id="openTwoBtn"]')
            button.click()
            time.sleep(1)
            
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            datas = soup.select('#openTrOne > tr')
            
            for tr in datas:
                values = [i+'년'] + [td.get_text(strip=True) for td in tr.find_all(["th", "td"])]
                year_data.append(values)
        
        driver.quit()
        return year_data
    
    def year_data_preprocessing(self, year_data):
        yaer_df = pd.DataFrame(data=year_data, columns=['recall_year','recall_month','kr_recall_car','kr_recall_num','us_recall_car','us_recall_num','total_recall_car','total_recall_num'])
    
        # dtype : str -> int
        for col in ['kr_recall_car','kr_recall_num','us_recall_car','us_recall_num','total_recall_car','total_recall_num']:
            yaer_df[col] = yaer_df[col].str.replace(',','').astype(int)
        return yaer_df

    def get_year_df(self):
        return self.recall_year_df

    def __init__(self):
        self.recall_year_df = self.year_data_preprocessing(self.get_year_data())

In [ ]:
# get_hyundai_faq_data.py
class Hyundai_faq:
    def get_hyundai_faq(self):
        URL = "https://www.hyundai.com/kr/ko/e/customer/center/faq"
    
        driver = webdriver.Chrome()
        driver.get(URL)
        # 리스트에 딕셔너리로 내용 저장
        hyundai_faq_data = []
        
        for page_id in range(4):
            for question_id in range(1,11):
                try:
                    #엘리먼트 찾기
                    faq_list = driver.find_element(By.CSS_SELECTOR, "div[data-v-28d34f54].list-wrap")
        
                    try:
                        # 플로팅 메뉴 제거
                        floating_menu = driver.find_element(By.CSS_SELECTOR, "div[data-v-1ea4ba2d].inner_wrap")
                        driver.execute_script("arguments[0].remove();", floating_menu)
                    except:
                        pass
                    faq_title = faq_list.find_elements(By.CSS_SELECTOR,f"div[data-id='{question_id}'] button.list-title")
                    driver.execute_script("arguments[0].scrollIntoView({block:'center'});",faq_title[0])
                    time.sleep(0.5)
        
                    #질문타이틀 클릭하기
                    faq_title[0].click()
                    time.sleep(0.5)
        
                    #질문타이틀 텍스트 받아오기
                    faq_question = faq_title[0].find_element(By.CSS_SELECTOR, "span.list-content[data-v-28d34f54]")
                    faq_question_text = faq_question.text
                    #print(faq_question_text)
        
                    #질문답변 텍스트 받아오기
                    faq_answer = driver.find_element(By.CLASS_NAME, "conts")
                    faq_answer_text = faq_answer.text
                    #print("전체 답변:", faq_answer_text)
                    
                    hyundai_faq_data.append({"hyundai_question": faq_question_text, "hyundai_answer": faq_answer_text})
        
                except TimeoutException:
                    print("Timed out waiting for page to load")
                except NoSuchElementException:
                    print("Could not find the element")
                except IndexError:
                    pass
        
            next_button = driver.find_element(By.CSS_SELECTOR, "button.btn-next")
            next_button.click()
            time.sleep(1)
        return hyundai_faq_data
            
    def hyundai_faq_preprocessing(self, hyundai_faq_data):
        hyundai_faq_df = pd.DataFrame(hyundai_faq_data)
        return hyundai_faq_df

    def get_hyundai_faq_df(self):
        return self.hyundai_faq_df

    def __init__(self):
        self.hyundai_faq_df = self.hyundai_faq_preprocessing(self.get_hyundai_faq())

In [74]:
# get_kia_faq_data.py
class Kia_Faq:
    def get_kia_faq(self):
        url = 'https://www.kia.com/kr/customer-service/center/faq'
        driver = webdriver.Chrome()
        driver.get(url)
        
        # 차량 구매 관련 FAQ 탭 클릭 (탭 변경)
        try:
            wait = WebDriverWait(driver, 10)
            purchase_tab = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tab-list"]/li[3]/button')))
            purchase_tab.click()
            time.sleep(1)  # 탭 변경 후 대기
        except Exception as e:
            print(" '차량 구매' 탭 클릭이 되지 않았습니다!!!:", e)
            driver.quit()
            exit()
        
        # 데이터 저장용 리스트
        kia_faq_data = []
        
        # 차량 구매 관련 FAQ 1~4페이지 크롤링 루프
        for page in range(1, 4):  # 1~3 페이지 반복
            # print(f"현재 {page}페이지 크롤링 중...")
        
            # 현재 페이지의 FAQ 목록 가져오기
            faq_items = driver.find_elements(By.CLASS_NAME, "cmp-accordion__item")
        
            for i, item in enumerate(faq_items):
                try:
                    # 질문 버튼 찾기
                    question_btn = item.find_element(By.CLASS_NAME, "cmp-accordion__button")
        
                    # 스크롤 동작 및 대기
                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", question_btn)
                    time.sleep(1.5) 
        
                    # 클릭해서 펼치기
                    question_btn.click()
                    time.sleep(1)  # 답변이 펼쳐질 시간 대기
        
                    # 질문 가져오기
                    question = question_btn.text.strip()
        
                    # 답변 긁어오기 cmp-accordion__panel
                    answer_panel = item.find_element(By.CLASS_NAME, "cmp-accordion__panel")
                    answer = answer_panel.text.strip()
        
                    kia_faq_data.append({"kia_question": question, "kia_answer": answer})
        
                    # print(f" {page}페이지 {i+1}번째 질문 크롤링 완료") 
        
                except Exception as e:
                    print(f" {page}페이지 {i+1}번째 질문 크롤링 오류:", e)
                    continue
        
            # 다음 페이지로 이동 (4페이지에서는 이동하지 않는다)
            if page < 4:
                try:
                    next_page_xpath = f'//*[@id="contents"]/div/div[3]/div/div/div[4]/div/ul/li[{page+1}]/a'
                    next_page_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, next_page_xpath)))
        
                    # 스크롤 이동 및 대기
                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", next_page_btn)
                    time.sleep(1.5) 
        
                    next_page_btn.click()
                    time.sleep(1)  # 페이지 로딩 대기
                except Exception as e:
                    print(f" {page}페이지에서 {page+1}페이지로 이동 실패:", e)
                    break
        
        # FAQ 페이지로 돌아가서 '신차' 검색
        try:
            # FAQ 페이지로 돌아가서 검색창에 "신차" 입력
            search_box_xpath = '//*[@id="searchName"]'  # 검색창의 XPATH
            search_box = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, search_box_xpath)))
            
            # 기존 텍스트 삭제 후 "신차" 입력
            search_box.clear()
            search_box.send_keys("신차")
            search_box.send_keys(Keys.ENTER)  # Enter 키 입력
            
            time.sleep(3)  # 검색 결과 로딩 대기
            #print("'신차' 검색 완료!")
        
            # 신차 검색 결과 크롤링 루프
            faq_items = driver.find_elements(By.CLASS_NAME, "cmp-accordion__item")
        
            for i, item in enumerate(faq_items):
                try:
                    # 질문 버튼 찾기
                    question_btn = item.find_element(By.CLASS_NAME, "cmp-accordion__button")
        
                    # 스크롤 동작 및 대기
                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", question_btn)
                    time.sleep(1.5) 
        
                    # 클릭해서 펼치기
                    question_btn.click()
                    time.sleep(1)  # 답변이 펼쳐질 시간 대기
        
                    # 질문 가져오기
                    question = question_btn.text.strip()
        
                    # 답변 긁어오기 cmp-accordion__panel
                    answer_panel = item.find_element(By.CLASS_NAME, "cmp-accordion__panel")
                    answer = answer_panel.text.strip()
        
                    kia_faq_data.append({"kia_question": question, "kia_answer": answer})
        
                    # print(f"신차 검색 결과 {i+1}번째 질문 크롤링 완료") 
        
                except Exception as e:
                    print(f"신차 검색 결과 {i+1}번째 질문 크롤링 오류:", e)
                    continue
        
        except Exception as e:
            print("'신차' 검색 오류:", e)
        driver.quit()
        return kia_faq_data
    
    def kia_faq_preprocessing(self, kia_faq_data):
        kia_faq_df = pd.DataFrame(kia_faq_data)
        
        # 필요없는 텍스트 제거
        kia_faq_df['kia_answer'] = kia_faq_df['kia_answer'].str.replace('바로가기 >', '')
        kia_faq_df['kia_answer'] = kia_faq_df['kia_answer'].str.replace('자세한 내용은 아래 링크를 클릭해 주세요.', '')
        kia_faq_df['kia_answer'] = kia_faq_df['kia_answer'].str.replace('EV & PBV 카탈로그/가격표 ', '')
        kia_faq_df['kia_answer'] = kia_faq_df['kia_answer'].str.replace('(구매보조금 지급 상한액은 아래표를 참고해주세요.)', '')
        kia_faq_df['kia_answer'] = kia_faq_df['kia_answer'].str.replace('시승 신청 바로가기 >', '')
        return kia_faq_df

    def get_kia_faq_df(self):
        return self.kia_faq_df

    def __init__(self):
        self.kia_faq_df = self.kia_faq_preprocessing(self.get_kia_faq())

In [ ]:
class Make_File:
    def make_file(self):
        if self.file_extensions == 'csv':
            df.to_csv(f'{self.file_name}.csv', index=False)
            print('생성완료')
        elif self.file_extensions == 'xlsx':
            df.to_excel(f'{self.file_name}.xlsx', index=False)
            print('생성완료')
        else:
            print('지원하지 않는 확장자명 입니다.')

    def __init__(self, df, file_extensions, file_name):
        self.df = df
        self.file_extensions = file_extensions
        self.file_name = file_name

In [24]:
res1 = get_manufacturer_data()

In [35]:
df1 = manufacturer_data_preprocessing(res1)
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 628 entries, 0 to 629
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   recall_year   628 non-null    object
 1   recall_month  628 non-null    object
 2   car_model     628 non-null    object
 3   car_num       628 non-null    int32 
dtypes: int32(1), object(3)
memory usage: 22.1+ KB


In [73]:
df1

,recall_year,recall_month,car_model,car_num
0,2022년,12월,비엠더블유코리아(주),33969
1,2022년,11월,비엠더블유코리아(주),43
2,2022년,10월,비엠더블유코리아(주),1866
3,2022년,09월,비엠더블유코리아(주),139
4,2022년,08월,비엠더블유코리아(주),19063
...,...,...,...,...
625,2024년,소계 +소계 -,케이지모빌리티커머셜 주식회사,52
626,2024년,08월,(주)바이크원(이륜),22
627,2024년,소계 +소계 -,(주)바이크원(이륜),22
628,2024년,01월,주식회사 이누리,873


In [36]:
res2 = get_year_data()

In [40]:
df2 = year_data_preprocessing(res2)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   recall_year       39 non-null     object
 1   recall_month      39 non-null     object
 2   kr_recall_car     39 non-null     int32 
 3   kr_recall_num     39 non-null     int32 
 4   us_recall_car     39 non-null     int32 
 5   us_recall_num     39 non-null     int32 
 6   total_recall_car  39 non-null     int32 
 7   total_recall_num  39 non-null     int32 
dtypes: int32(6), object(2)
memory usage: 1.6+ KB


In [42]:
res3 = get_hyundai_faq()

In [43]:
df3 = hyundai_faq_preprocessing(res3)
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   hyundai_question  34 non-null     object
 1   hyundai_answer    34 non-null     object
dtypes: object(2)
memory usage: 672.0+ bytes


In [51]:
res4 = get_kia_faq()

신차 검색 결과 1번째 질문 크롤링 완료
신차 검색 결과 2번째 질문 크롤링 완료


In [67]:
df4 = kia_faq_preprocessing(res4)
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   kia_question  32 non-null     object
 1   kia_answer    32 non-null     object
dtypes: object(2)
memory usage: 640.0+ bytes
